In [36]:
import requests
import lxml
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm, trange
import os
import pdfplumber


def read_pdf(file_name):
    with pdfplumber.open(f"./pdf_hwp/{file_name}") as pdf:
        pages = pdf.pages
        text = ""
        for page in pages:
            text += page.extract_text()
        if len(text) < 10:
            text = f"{file_name} 한글 인식 불가 ! OCR 필요"
    return text


# file_id = '1138672129786_104849.pdf'
# file_name = 'down.pdf'
def download_file(file_id, file_name=None):
    if not file_name:
        return
    download_url = f"https://file.scourt.go.kr//AttachDownload?&name=downForm&file={file_id}&path=003&downFile={file_name}"

    with open(f"./pdf_hwp/{file_name}", "wb") as file:
        response = requests.get(download_url)
        file.write(response.content)


def make_csv_file(body_content, file_name=None):
    if not file_name:
        return
    file_name = re.sub(r".pdf", r".csv", file_name)
    pd_body_content = pd.DataFrame(body_content)
    pd_body_content.to_csv(path_or_buf=f"./pdf_hwp/{file_name}", index=False)


for i in tqdm(range(20000, 20010)):
    base_url = "https://busan.scourt.go.kr/dcboard/new/DcNewsViewAction.work?&gubun=44&cbub_code=000410&searchWord=&pageIndex=1"
    params = {"seqnum": i}
    print(i)
    resp = requests.get(base_url, params=params)
    soup = BeautifulSoup(resp.content, "lxml")
    tbody_list = []
    tbody = soup.find("tbody")
    title = tbody.find("td", class_="title").text
    print(title)
    court = tbody.select_one("tr:nth-child(2) > td:nth-child(2)").text
    print(court)
    downloads = tbody.select("tr:nth-child(3) > td > a")
    print((downloads))
    url_regex = re.compile(r"\'(.*\.pdf)\'\,\'(.*.pdf)\'")
    body = tbody.find("div", class_="view_content").getText()
    print(body)
    file_list = []
    for download in downloads:
        file_id = url_regex.search(str(download)).group(1)
        file_name = url_regex.search(str(download)).group(2)
        file_list.append(file_name)

        print(file_id)
        print(file_name)
        download_file(file_id, file_name)

    body_content = [
        {
            "제목": title,
            "법원": court,
            "내용": body,
            "file_name": file_list,
            "file_body": None,
        }
    ]
    text_list = []
    for file_name in file_list:
        text_list.append(read_pdf(file_name))

    body_content[0]["file_body"] = text_list

    make_csv_file(body_content, file_name)


  0%|          | 0/10 [00:00<?, ?it/s]

20000
[형사] 사기 등 사건(전주지법, 2017노1590)
전주지방법원
[<a href="javascript:download('1531100227238_103707.pdf','전주지방법원_2017노1590.pdf')" title="전주지방법원_2017노1590.pdf">전주지방법원_2017노1590.pdf</a>]


피고인들이 대한민국공예품대전에 작품을 출품하여 대상을 수상하였으나, 출품한 작품 중 직접 제작하지 않은 부분이 있음에도 마치 모두 직접 제작한 것처럼 출품한 사안에 대하여 사기죄, 위계공무집행방해죄, 업무방해죄를 인정한 판결입니다.

1531100227238_103707.pdf
전주지방법원_2017노1590.pdf


 10%|█         | 1/10 [00:00<00:08,  1.00it/s]

20001
[형사] 의료법위반사건(전주지법, 2017노1766)
전주지방법원
[<a href="javascript:download('1531100301773_103821.pdf','전주지방법원_2017노1766.pdf')" title="전주지방법원_2017노1766.pdf">전주지방법원_2017노1766.pdf</a>]


의사가 다른 병원에서 정기적으로 의료행위를 한 경우 의료법위반죄로 처벌대상이 되는지에 대해서 1심에서는 무죄를, 2심에서는 유죄를 인정한 사안으로, 현재 이 사건은 피고인의 상고로 대법원 심리가 예정되어 있습니다. 1, 2심의 논리를 비교하여 보고 나름의 결론을 생각해 보시면 법률해석의 재미를 느끼실 수 있을 것입니다.

1531100301773_103821.pdf
전주지방법원_2017노1766.pdf


 20%|██        | 2/10 [00:01<00:07,  1.12it/s]

...

 80%|████████  | 8/10 [00:06<00:01,  1.19it/s]

20008
[형사] 전 동거녀와 사귀는 남성에 대한 살인미수 사건(창원지방법원 2018고합9)
창원지방법원
[<a href="javascript:download('1531101664723_110104.pdf','2018고합9_검수완료.pdf')" title="2018고합9_검수완료.pdf">2018고합9_검수완료.pdf</a>]


★창원지방법원 2018. 4. 17. 선고 2018고합9 판결
  
제목 [형사] 전 동거녀와 사귀는 남성에 대한 살인미수 사건
  
[사안의 개요] 
피고인이 자신의 전 동거녀 집에 흉기를 들고 침입한 후 전 동거녀를 때리고 전 동거녀와 사귀고 있는 남성을 칼로 찔러 살해하려다 미수에 그쳐 징역형을 선고받은 사안
  

1531101664723_110104.pdf
2018고합9_검수완료.pdf


 90%|█████████ | 9/10 [00:07<00:00,  1.24it/s]

20009
[형사] 지하주차장에서 혼자 있는 피해자를 강제추행한 사건(창원지방법원 2018고단1061)
창원지방법원
[<a href="javascript:download('1531101734727_110214.pdf','2018고단1061_검수완료.pdf')" title="2018고단1061_검수완료.pdf">2018고단1061_검수완료.pdf</a>]

  ★창원지방법원 2018. 6. 28. 선고 2018고단1061 판결
  
제목 [형사] 지하주차장에서 혼자 있는 피해자를 강제추행한 사건
  
[사안의 개요] 
피고인이 상남동 빌딩 지하주차장에서 피해자 차량의 창문을 두드려, 문을 열고 "왜요?"라고 묻는 피해자에게 뽀뽀를 하고 치마 속에 손을 넣는 등 추행하였다가 징역형의 집행유예를 선고받은 사안
  
  

1531101734727_110214.pdf
2018고단1061_검수완료.pdf


100%|██████████| 10/10 [00:07<00:00,  1.25it/s]


In [18]:


# print(read_pdf('2005허5860.pdf'))
# with open('./pdf_hwp/eee.txt', 'w') as f:
#     text_desc = read_pdf('2005허5860.pdf')
#     f.write(text_desc)
    


2005허5860.pdf 한글 인식 불가 ! OCR 필요
